In [7]:
import src.mylib
import os
KEY1=src.mylib.getKey(os.path.join('src','key.properties'))
print 'gokr=',KEY1['gokr']
print 'dataseoul=',KEY1['dataseoul']

gokr= sXf66NX9wb7bv6O7tj%2BGUQwoC8se7eLri6DeEShKqQAj19JzR8xl29bBfbN1L0%2BqGdQUtDRTn1hm6wSIfle4Tg%3D%3D
dataseoul= 674d7a584e6b656c35317057737a5a


* 열린데이터 홈페이지에가서 상세설명을 보고 그 양식에 맞춰서 KEY, TYPE, SERVICE, START_INDEX, END_INDEX, LINE_NUM 이라는 변수명을 알아 온다.

In [22]:
%%writefile src/ds_open_dataseoul_ex.py
# coding:utf-8
import os
import requests
import urlparse
import mylib
import lxml
import lxml.etree
import StringIO

def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    key=mylib.getKey(keyPath)
    # (1) make params with resource IDs
    KEY=key['dataseoul']
    TYPE='json'
    SERVICE='SearchSTNBySubwayLineService'
    START_INDEX=str(1)
    END_INDEX=str(10)
    LINE_NUM=str(2)
    #params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,LINE_NUM)
    params='/'+KEY+'/'+TYPE+'/'+SERVICE+'/'+START_INDEX+'/'+END_INDEX+'/'+LINE_NUM
    # (2) make a full url
    _url='http://openAPI.seoul.go.kr:8088/'
    url=urlparse.urljoin(_url,params)
    # (3) get data
    data=requests.get(url).text
    #print data[:300]
    tree=lxml.etree.fromstring(data.encode('utf-8'))
    nodes=tree.xpath('//STATION_NM')
    for node in nodes:
        print node.text
if __name__ == "__main__":
    doIt()

Overwriting src/ds_open_dataseoul_ex.py


In [3]:
pwd

u'C:\\Users\\400T6B\\Code\\s_201511138'

In [9]:
import requests
import urlparse
print os.path.join(os.getcwd(),'src','key.properties')
_url='http://openAPI.seoul.go.kr:8088/'
KEY='dfadf'
TYPE='json'
SERVICE='SearchSTNBySubwayLineService'
START_INDEX=str(1)
END_INDEX=str(10)
LINE_NUM=str(2)
params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,LINE_NUM)

C:\Users\Y\Code\s_201511138\src\key.properties


In [39]:
import sys #src.mylib을 import해서 써주기위한 설정
del sys.modules['src.mylib']

In [45]:
import os
import requests
import urlparse
import lxml
import lxml.etree
import StringIO

def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    #key=mylib.getKey(keyPath)
    # (1) make params with resource IDs
    KEY='674d7a584e6b656c35317057737a5a'
    TYPE='json'
    SERVICE='SearchSTNBySubwayLineService'
    START_INDEX=str(1)
    END_INDEX=str(10)
    LINE_NUM=str(2)
    #params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,LINE_NUM)
    params='/'+KEY+'/'+TYPE+'/'+SERVICE+'/'+START_INDEX+'/'+END_INDEX+'/'+LINE_NUM
    # (2) make a full url
    _url='http://openAPI.seoul.go.kr:8088/'
    url=urlparse.urljoin(_url,params)
    # (3) get data
    data=requests.get(url).text
    #print data[:300]
    tree=lxml.etree.fromstring(data.encode('utf-8'))
    nodes=tree.xpath('//STATION_NM')
    
    for node in nodes:
        print node.text
        
if __name__ == "__main__":
    doIt()

XMLSyntaxError: Start tag expected, '<' not found, line 1, column 1 (line 1)

In [49]:
%%writefile src/ds_open_subway.py
import os
import urlparse
import requests
import lxml
import lxml.etree
import StringIO
import mylib
import json
from pymongo import MongoClient

Client = MongoClient('localhost:27017')
_db=Client['ds_open_subwayDb'] #db created by mongo
_table=_db['db_open_subwayTable'] #collection
#db=Client.ds_rest_subwayPassengers
        
def saveJson(_fname,_data):
    import io
    with io.open(_fname, 'a', encoding='utf8') as json_file:
        _j=json.dumps(_data, json_file, ensure_ascii=False, encoding='utf8')
        json_file.write(_j+"\n")

def readJson(_fname):
    for line in open(_fname, 'r').readlines():
        _j=json.loads(line)
        #print _j['id'],_j['text']
        print _j['id']        
def saveDB(_data):
    _table.insert_one(_data)
def readDB():
    for tweet in _table.find():
        print tweet['id'],tweet['text']

def saveFile(_fname,_data):
    fp=open(_fname,'a')
    fp.write(_data+"\n")
    
def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    key=mylib.getKey(keyPath)
    KEY=str(key['dataseoul'])
    TYPE='xml'
    SERVICE='SearchSTNBySubwayLineService'
    START_INDEX=str(1)
    END_INDEX=str(10)
    LINE_NUM=str(2)

    _maxIter=20
    _iter=0
    _jfname='src/ds_open_subway.json'
    params='/'+KEY+'/'+TYPE+'/'+SERVICE+'/'+START_INDEX+'/'+END_INDEX+'/'+LINE_NUM

    _url='http://openAPI.seoul.go.kr:8088/'
    url=urlparse.urljoin(_url,params)
    data=requests.get(url).text
    tree=lxml.etree.fromstring(data.encode('utf-8'))
    nodes=tree.xpath('//STATION_NM')
    _json=nodes.json()
    print _json
    saveJson(_jfname,_json)
    saveDB(_json)
    START_INDEX+=5
    
    for node in nodes:
        print node.text

    
if __name__ == "__main__":
    doIt()

Overwriting src/ds_open_subway.py


In [47]:
!python src\ds_open_subway.py

Traceback (most recent call last):
  File "src\ds_open_subway.py", line 30, in <module>
    doIt()
  File "src\ds_open_subway.py", line 24, in doIt
    tree=lxml.etree.fromstring(data.encode('utf-8'))
  File "src\lxml\lxml.etree.pyx", line 3213, in lxml.etree.fromstring (src\lxml\lxml.etree.c:78993)
  File "src\lxml\parser.pxi", line 1848, in lxml.etree._parseMemoryDocument (src\lxml\lxml.etree.c:118324)
  File "src\lxml\parser.pxi", line 1736, in lxml.etree._parseDoc (src\lxml\lxml.etree.c:117004)
  File "src\lxml\parser.pxi", line 1102, in lxml.etree._BaseParser._parseDoc (src\lxml\lxml.etree.c:111248)
  File "src\lxml\parser.pxi", line 595, in lxml.etree._ParserContext._handleParseResultDoc (src\lxml\lxml.etree.c:105092)
  File "src\lxml\parser.pxi", line 706, in lxml.etree._handleParseResult (src\lxml\lxml.etree.c:106800)
  File "src\lxml\parser.pxi", line 646, in lxml.etree._raiseParseError (src\lxml\lxml.etree.c:105946)
  File "<string>", line 0
lxml.etree.XMLSyntaxError


In [4]:
!dir src

 C 드라이브의 볼륨: Windows
 볼륨 일련 번호: A065-E724

 C:\Users\Y\Code\s_201511138\src 디렉터리

2017-04-24  오후 06:56    <DIR>          .
2017-04-24  오후 06:56    <DIR>          ..
2017-04-16  오후 08:55               253 ds234_web_crawl_1.py
2017-04-16  오후 08:55               436 ds234_web_crawl_5.py
2017-04-16  오후 08:55               640 ds234_web_crawl_6.py
2017-04-16  오후 08:55               578 ds2_web_crawl_3.py
2017-04-16  오후 08:55               529 ds2_web_crawl_4.py
2017-04-24  오후 06:56               766 ds_open_dataseoul_ex.py
2017-04-17  오후 01:39               165 ds_save_mongo_5.js
2017-04-17  오후 01:56               209 ds_save_mongo_6.js
2017-04-17  오후 02:06               296 ds_save_mongo_6.py
2017-04-16  오후 08:55                79 ds_save_mongo_test.js
2017-04-24  오후 12:20                73 ds_save_persons.json
2017-04-24  오후 12:21                73 ds_save_persons1.json
2017-04-16  오후 08:55               374 ds_web_crawl_ieee.py
2017-04-23  오후 01:20               822 ds_web_crawl_naverMus

* ! : 명령창에서 다음을 실행시켜서 ipython note북상에서 보여주십쇼

In [10]:
!python src/ds_open_dataseoul_ex.py

Traceback (most recent call last):
  File "src/ds_open_dataseoul_ex.py", line 27, in <module>
    doIt()
  File "src/ds_open_dataseoul_ex.py", line 24, in doIt
    print data[:300]
UnicodeEncodeError: 'ascii' codec can't encode characters in position 94-95: ordinal not in range(128)


* 공공데이터 포탈은 직접들어가서 doc파일을 다운받아서 그것을 열어보면 필수 요청 변수명을 알 수 있다. 

In [14]:
%%writefile src/ds_open_gokr_ex.py
# coding: utf-8
import os
import requests
import urlparse
import urllib
import mylib

def doIt():
    # (1) service + operation
    SERVICE='ArpltnInforInqireSvc'
    OPERATION_NAME='getMinuDustFrcstDspth'
    params1=SERVICE+'/'+OPERATION_NAME
    # (2) query params encoding
    _d=dict()
    _d['dataTerm']='month'
    params2 = urllib.urlencode(_d)
    # (3) add my service key
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    key=mylib.getKey(keyPath)
    keygokr=key['gokr'] # keygokr='8Bx4C1%2B...'
    params=params1+'?'+'serviceKey='+keygokr+'&'+params2
    # (4) make a full url
    _url='http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc'
    url=urlparse.urljoin(_url,params)
    # (5) get data
    data=requests.get(url).text
    print data[:300]

if __name__ == "__main__":
    doIt()

Overwriting src/ds_open_gokr_ex.py


In [15]:
!python src\ds_open_dataseoul_ex.py

Traceback (most recent call last):
  File "src\ds_open_dataseoul_ex.py", line 27, in <module>
    doIt()
  File "src\ds_open_dataseoul_ex.py", line 24, in doIt
    print data[:300]
UnicodeEncodeError: 'ascii' codec can't encode characters in position 94-95: ordinal not in range(128)
